In [1]:
import tensorflow
tensorflow.__version__

'1.15.0'

In [9]:
weightss = [0.05,0.4,0.05,0.25,0.25]
def weighted_categorical_crossentropy(weights):
    def wcce(y_true, y_pred):
        Kweights = K.constant(weights)
        if not K.is_tensor(y_pred): y_pred = K.constant(y_pred)
        y_true = K.cast(y_true, y_pred.dtype)
        return K.sparse_categorical_crossentropy(y_true, y_pred) * K.sum(y_true * Kweights, axis=-1)
    return wcce

In [10]:
def FocalLoss11(targets, inputs, alpha=.5, gamma=2):    
    #targets = K.cast(targets, 'float32')
    BCE = K.sparse_categorical_crossentropy(targets, inputs)
    BCE_EXP = K.exp(-BCE)
    focal_loss = K.mean(alpha * K.pow((1-BCE_EXP), gamma) * BCE)
    return focal_loss

In [31]:
from sklearn.utils import class_weight


def FocalLoss_weighted(targets, inputs, gamma=2): 
    #class_weights = class_weight.compute_class_weight('balanced',
     #                                            np.unique(inputs[0,:,0]),
      #                                           inputs[0,:,0])
    alpha = tf.constant([[.1],[.6],[.1],[.3],[.3]], dtype=tf.float32)
    #alpha = tf.constant([class_weights], dtype=tf.float32)
    #alpha=tf.constant([.1,.3,.1,.25,.25], dtype=tf.float32)
    #targets = K.cast(targets, 'float32')
    BCE = K.sparse_categorical_crossentropy(targets, inputs)
    BCE_EXP = K.exp(-BCE)
    focal_loss = K.mean(tf.matmul(alpha , K.pow((1-BCE_EXP), gamma) * BCE))
    return focal_loss

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

# focal_loss20fold

In [108]:
%cd C:\Users\Admin\Desktop\ebrahi
from keras import backend as K
import models 
import ASSC_bach_generetor
model=model_TCNN_CRF()
model.compile(optimizers.Adam(0.001,amsgrad=True),loss=FocalLoss11, metrics=[crf_marginal_accuracy])


#/pure_aa
import gc
import random
base_path = "20fold"    
files = sorted(glob(os.path.join(base_path, "*.npz")))
j=0
gt_t=[]
preds_t=[]
preds_cat_cat=[]
kk=0
for i in range(20):
        ids = sorted(list(set([x.split("/")[-1][:5] for x in files])))
        ids=[x for x in files if x.split("/")[-1][:5] in ids]
        if kk<38:
            d=ids[kk:kk+2]
            idea=ids[kk+2:]+ids[0:kk]
            val=[idea.pop(int(kk/2)),idea.pop(int(kk/2)+1)]
            val_dict={k: np.load(k) for k in val}
            train_dict = {k: np.load(k) for k in idea}    
            test_dict = {k: np.load(k) for k in d}
        else :
            idea=ids[:38]
            d=ids[38]
            val=[idea.pop(int(kk/2)),idea.pop(int(kk/2)+1)]
            val_dict={k: np.load(k) for k in val}
            train_dict = {k: np.load(k) for k in idea}
            test_dict = {k: np.load(k) for k in [d]}
        kk=kk+2
        file_path = "bala.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='acc',verbose=1, save_best_only=True, mode='max')
        early = EarlyStopping(monitor='acc', mode="max", patience=20, verbose=1)
        redonplat = ReduceLROnPlateau(monitor='acc', mode="max", factor=0.1, patience=4, verbose=2)#, factor=0.2
        callbacks_list = [checkpoint, early ,redonplat]
        
        model = get_base_model()
        
        #model.fit_generator(gen_balance(train_dict),validation_data=gen_j(val_dict),epochs=1,verbose=2,
         #                            steps_per_epoch=1000,validation_steps=300, callbacks=callbacks_list) 

        K.clear_session()
        del model
        
        gc.collect()
        file_path = "model1.h5"
        
        checkpoint = ModelCheckpoint(file_path, monitor='val_crf_marginal_accuracy',verbose=1, save_best_only=True, mode='max')
        early = EarlyStopping(monitor='val_crf_marginal_accuracy', mode="max", patience=20, verbose=1)
        redonplat = ReduceLROnPlateau(monitor='val_crf_marginal_accuracy', mode="max", factor=0.5, patience=4, verbose=2)#, factor=0.2
        callbacks_list = [checkpoint, early ,redonplat]
        model = get_base_model()
        model.fit_generator(gen_generator(train_dict),validation_data=gen_j(val_dict),epochs=25,verbose=2,
                                     steps_per_epoch=1000,validation_steps=300, callbacks=callbacks_list)
        
        K.clear_session()
        del model
        gc.collect()
        file_path = "model0.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_crf_marginal_accuracy',verbose=1, save_best_only=True, mode='max')
        early = EarlyStopping(monitor='val_crf_marginal_accuracy', mode="max", patience=20, verbose=1)
        redonplat = ReduceLROnPlateau(monitor='val_crf_marginal_accuracy', mode="max", factor=0.5, patience=4, verbose=2)#, factor=0.2
        callbacks_list = [checkpoint, early ,redonplat]
        model = get_base_model()
        #model.load_weights('bala.h5')
        model.fit_generator(gen_j(train_dict),validation_data=gen_j(val_dict),epochs=25,verbose=2,
                                      steps_per_epoch=1000,validation_steps=300, callbacks=callbacks_list) 
        
        K.clear_session()
        del model
        gc.collect()
        
        h=0
        j=0
        gg=[]
        g=[]
        p=[]
        p_cat=[]
        for record in tqdm(test_dict):
                all_rows = test_dict[record]['x']
                
              
                l=0
                j=j+1
                for batch_hyp in chunker(range(all_rows.shape[0])):
                        l=l+1
                        X1 = all_rows[min(batch_hyp):max(batch_hyp)+1, ...]
                        Y1= test_dict[record]['y'][min(batch_hyp):max(batch_hyp)+1]
                        
                        #######################################
                        X1 =rescale_array2(X1)
                        X1 = np.expand_dims(X1, 0)
           
                        model = get_base_model()
                        model.load_weights('model1.h5')
                        Y_pred1 = model.predict([X1,X1])
                        model.load_weights('model0.h5')
                        Y_pred0 = model.predict([X1,X1])
                      
                        Y_pred=Y_pred1 +Y_pred0
                        Y_pred = Y_pred.argmax(axis=-1).ravel().tolist()
                        
                        gt_t += Y1.ravel().tolist()
                        preds_t += Y_pred
                        g+=Y1.ravel().tolist()
                        p+=Y_pred
      #####
        print(test_dict)
        f1 = f1_score(g, p, average="macro")
        print("Seq Test f1 score (deep): %s "% f1)
        acc = accuracy_score(g, p)
        print("Seq Test accuracy score(deeo) : %s "% acc)
        print(classification_report(g, p))
        cnf_matrix = confusion_matrix(g, p)
        np.set_printoptions(precision=2)
        print(cnf_matrix)
        from keras import backend as K 
        K.clear_session()
        del model
f1 = f1_score(gt_t, preds_t, average="macro")
print("Seq Test f1 score : %s "% f1)
acc = accuracy_score(gt_t, preds_t)
print("Seq Test accuracy score : %s "% acc)
print(classification_report(gt_t, preds_t))
class_names=np.array([["W"], ["S1"],["S2"],["S3"],["R"]])
# Compute confusion matrix
import itertools
from sklearn.metrics import confusion_matrix



cnf_matrix = confusion_matrix(gt_t, preds_t)
print(cnf_matrix)

C:\Users\Admin\Desktop\ebrahi
Epoch 1/25
 - 246s - loss: 0.4325 - crf_marginal_accuracy: 0.5651 - val_loss: 0.1620 - val_crf_marginal_accuracy: 0.7008

Epoch 00001: val_crf_marginal_accuracy improved from -inf to 0.70080, saving model to model1.h5
Epoch 2/25
 - 243s - loss: 0.2211 - crf_marginal_accuracy: 0.6395 - val_loss: 0.7285 - val_crf_marginal_accuracy: 0.7114

Epoch 00002: val_crf_marginal_accuracy improved from 0.70080 to 0.71143, saving model to model1.h5
Epoch 3/25
 - 239s - loss: 0.1993 - crf_marginal_accuracy: 0.6570 - val_loss: 0.1060 - val_crf_marginal_accuracy: 0.7277

Epoch 00003: val_crf_marginal_accuracy improved from 0.71143 to 0.72767, saving model to model1.h5
Epoch 4/25
 - 238s - loss: 0.1909 - crf_marginal_accuracy: 0.7017 - val_loss: 0.0947 - val_crf_marginal_accuracy: 0.7387

Epoch 00004: val_crf_marginal_accuracy improved from 0.72767 to 0.73873, saving model to model1.h5
Epoch 5/25
 - 245s - loss: 0.1717 - crf_marginal_accuracy: 0.7321 - val_loss: 0.1385 - va

100%|██████████| 2/2 [04:08<00:00, 124.07s/it]


{'20fold\\SC4001E0.npz': <numpy.lib.npyio.NpzFile object at 0x0000028E9E064708>, '20fold\\SC4002E0.npz': <numpy.lib.npyio.NpzFile object at 0x0000028E9E064B48>}
Seq Test f1 score (deep): 0.7961537098391454 
Seq Test accuracy score(deeo) : 0.8572154471544715 
              precision    recall  f1-score   support

           0       0.93      0.89      0.91       371
           1       0.55      0.38      0.45       117
           2       0.84      0.88      0.86       623
           3       0.93      0.84      0.88       517
           4       0.80      0.98      0.88       340

    accuracy                           0.86      1968
   macro avg       0.81      0.79      0.80      1968
weighted avg       0.86      0.86      0.85      1968

[[329  21   0   1  20]
 [ 21  45  14   0  37]
 [  1  16 547  32  27]
 [  2   0  81 433   1]
 [  0   0   7   0 333]]
Epoch 1/25
 - 309s - loss: 0.4050 - crf_marginal_accuracy: 0.5416 - val_loss: 0.5268 - val_crf_marginal_accuracy: 0.5760

Epoch 00001: v

100%|██████████| 2/2 [05:24<00:00, 162.45s/it]


{'20fold\\SC4011E0.npz': <numpy.lib.npyio.NpzFile object at 0x0000028E9E0646C8>, '20fold\\SC4012E0.npz': <numpy.lib.npyio.NpzFile object at 0x0000028E9E064108>}
Seq Test f1 score (deep): 0.8304234229036835 
Seq Test accuracy score(deeo) : 0.8807339449541285 
              precision    recall  f1-score   support

           0       0.91      0.87      0.89       319
           1       0.73      0.58      0.65       201
           2       0.95      0.91      0.93      1222
           3       0.75      0.89      0.81       201
           4       0.81      0.97      0.88       346

    accuracy                           0.88      2289
   macro avg       0.83      0.84      0.83      2289
weighted avg       0.88      0.88      0.88      2289

[[ 277   26    1   13    2]
 [  20  116   32    1   32]
 [   6   13 1110   47   46]
 [   0    0   22  179    0]
 [   2    3    7    0  334]]
Epoch 1/25
 - 293s - loss: 0.5360 - crf_marginal_accuracy: 0.4727 - val_loss: 0.4057 - val_crf_marginal_accurac

...

100%|██████████| 2/2 [08:06<00:00, 243.25s/it]


{'20fold\\SC4191E0.npz': <numpy.lib.npyio.NpzFile object at 0x0000028F450A5688>, '20fold\\SC4192E0.npz': <numpy.lib.npyio.NpzFile object at 0x0000028F450A5308>}
Seq Test f1 score (deep): 0.7129811794959213 
Seq Test accuracy score(deeo) : 0.7999288002847988 
              precision    recall  f1-score   support

           0       0.73      0.89      0.80       564
           1       0.36      0.31      0.33       190
           2       0.90      0.87      0.89      1267
           3       0.65      0.99      0.79       170
           4       0.87      0.67      0.76       618

    accuracy                           0.80      2809
   macro avg       0.70      0.75      0.71      2809
weighted avg       0.81      0.80      0.80      2809

[[ 502   19   38    0    5]
 [  74   59   33    0   24]
 [   7   38 1101   90   31]
 [   0    0    2  168    0]
 [ 107   50   44    0  417]]
Epoch 1/25
 - 330s - loss: 0.4273 - crf_marginal_accuracy: 0.5516 - val_loss: 0.3100 - val_crf_marginal_accurac

100%|██████████| 1/1 [01:35<00:00, 95.96s/it]

{'20fold\\SC4231E0.npz': <numpy.lib.npyio.NpzFile object at 0x0000028F0A7CD108>}
Seq Test f1 score (deep): 0.8205738269822612 
Seq Test accuracy score(deeo) : 0.877431906614786 
              precision    recall  f1-score   support

           0       0.91      0.90      0.90       155
           1       0.49      0.51      0.50        57
           2       0.93      0.87      0.90       497
           3       0.90      0.93      0.92       147
           4       0.83      0.95      0.88       172

    accuracy                           0.88      1028
   macro avg       0.81      0.83      0.82      1028
weighted avg       0.88      0.88      0.88      1028

[[139   6   7   0   3]
 [ 12  29   7   0   9]
 [  2  24 434  15  22]
 [  0   0  10 137   0]
 [  0   0   9   0 163]]
Seq Test f1 score : 0.751526717197366 
Seq Test accuracy score : 0.8192304055970502 
              precision    recall  f1-score   support

           0       0.88      0.84      0.86      8285
           1       0.47

NameError: name 'plt' is not defined

In [110]:
print(confusion_matrix(gt_t, preds_t))

[[ 6980   520   374    26   385]
 [  566   976   683    14   565]
 [  115   392 15325  1087   880]
 [   13     5   912  4757    16]
 [  248   187   659     1  6622]]


# focal_weigted_loss20fold

In [38]:
%cd C:\Users\Admin\Desktop\ebrahi
from keras import backend as K
import models 
model=model_TCNN_CRF()
model.compile(optimizers.Adam(0.001,amsgrad=True),loss=FocalLoss_weighted, metrics=[crf_marginal_accuracy])

#/pure_aa
import gc
import random
base_path = "20fold"    
files = sorted(glob(os.path.join(base_path, "*.npz")))
j=0








gt_t=[]
preds_t=[]
preds_cat_cat=[]
kk=0
for i in range(20):
        ids = sorted(list(set([x.split("/")[-1][:5] for x in files])))
        ids=[x for x in files if x.split("/")[-1][:5] in ids]
        if kk<38:
            d=ids[kk:kk+2]
            idea=ids[kk+2:]+ids[0:kk]
            val=[idea.pop(int(kk/2)),idea.pop(int(kk/2)+1)]
            val_dict={k: np.load(k) for k in val}
            train_dict = {k: np.load(k) for k in idea}    
            test_dict = {k: np.load(k) for k in d}
        else :
            idea=ids[:38]
            d=ids[38]
            val=[idea.pop(int(kk/2)),idea.pop(int(kk/2)+1)]
            val_dict={k: np.load(k) for k in val}
            train_dict = {k: np.load(k) for k in idea}
            test_dict = {k: np.load(k) for k in [d]}
        kk=kk+2
        file_path = "bala.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='acc',verbose=1, save_best_only=True, mode='max')
        early = EarlyStopping(monitor='acc', mode="max", patience=20, verbose=1)
        redonplat = ReduceLROnPlateau(monitor='acc', mode="max", factor=0.1, patience=4, verbose=2)#, factor=0.2
        callbacks_list = [checkpoint, early ,redonplat]
        
        model = get_base_model()
        
        #model.fit_generator(gen_balance(train_dict),validation_data=gen_j(val_dict),epochs=1,verbose=2,
         #                            steps_per_epoch=1000,validation_steps=300, callbacks=callbacks_list) 

        K.clear_session()
        del model
        
        gc.collect()
        file_path = "model1.h5"
        
        checkpoint = ModelCheckpoint(file_path, monitor='val_crf_marginal_accuracy',verbose=1, save_best_only=True, mode='max')
        early = EarlyStopping(monitor='val_crf_marginal_accuracy', mode="max", patience=20, verbose=1)
        redonplat = ReduceLROnPlateau(monitor='val_crf_marginal_accuracy', mode="max", factor=0.5, patience=4, verbose=2)#, factor=0.2
        callbacks_list = [checkpoint, early ,redonplat]
        model = get_base_model()
        model.fit_generator(gen_generator(train_dict),validation_data=gen_j(val_dict),epochs=25,verbose=2,
                                     steps_per_epoch=1000,validation_steps=300, callbacks=callbacks_list)
        
        K.clear_session()
        del model
        gc.collect()
        file_path = "model0.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_crf_marginal_accuracy',verbose=1, save_best_only=True, mode='max')
        early = EarlyStopping(monitor='val_crf_marginal_accuracy', mode="max", patience=20, verbose=1)
        redonplat = ReduceLROnPlateau(monitor='val_crf_marginal_accuracy', mode="max", factor=0.5, patience=4, verbose=2)#, factor=0.2
        callbacks_list = [checkpoint, early ,redonplat]
        model = get_base_model()
        #model.load_weights('bala.h5')
        model.fit_generator(gen_j(train_dict),validation_data=gen_j(val_dict),epochs=25,verbose=2,
                                      steps_per_epoch=1000,validation_steps=300, callbacks=callbacks_list) 
        
        K.clear_session()
        del model
        gc.collect()
        
        h=0
        j=0
        gg=[]
        g=[]
        p=[]
        p_cat=[]
        for record in tqdm(test_dict):
                all_rows = test_dict[record]['x']
                
              
                l=0
                j=j+1
                for batch_hyp in chunker(range(all_rows.shape[0])):
                        l=l+1
                        X1 = all_rows[min(batch_hyp):max(batch_hyp)+1, ...]
                        Y1= test_dict[record]['y'][min(batch_hyp):max(batch_hyp)+1]
                        #######################################
                        X1 =rescale_array2(X1)
                        X1 = np.expand_dims(X1, 0)
                        model = get_base_model()
                        model.load_weights('model1.h5')
                        Y_pred1 = model.predict([X1,X1])
                        model.load_weights('model0.h5')
                        Y_pred0 = model.predict([X1,X1])
                        Y_pred=Y_pred1 +Y_pred0
                        Y_pred = Y_pred.argmax(axis=-1).ravel().tolist()
                        gt_t += Y1.ravel().tolist()
                        preds_t += Y_pred
                        g+=Y1.ravel().tolist()
                        p+=Y_pred
      #####
        print(test_dict)
        f1 = f1_score(g, p, average="macro")
        print("Seq Test f1 score (deep): %s "% f1)
        acc = accuracy_score(g, p)
        print("Seq Test accuracy score(deeo) : %s "% acc)
        print(classification_report(g, p))
        cnf_matrix = confusion_matrix(g, p)
        np.set_printoptions(precision=2)
        print(cnf_matrix)
        from keras import backend as K 
        K.clear_session()
        del model
f1 = f1_score(gt_t, preds_t, average="macro")
print("Seq Test f1 score : %s "% f1)
acc = accuracy_score(gt_t, preds_t)
print("Seq Test accuracy score : %s "% acc)
print(classification_report(gt_t, preds_t))
class_names=np.array([["W"], ["S1"],["S2"],["S3"],["R"]])
# Compute confusion matrix
import itertools
from sklearn.metrics import confusion_matrix
cnf_matrix = confusion_matrix(gt_t, preds_t)
print(cnf_matrix)


C:\Users\Admin\Desktop\ebrahi
Epoch 1/25
 - 233s - loss: 0.2534 - crf_marginal_accuracy: 0.5799 - val_loss: 0.1126 - val_crf_marginal_accuracy: 0.6860

Epoch 00001: val_crf_marginal_accuracy improved from -inf to 0.68600, saving model to model1.h5
Epoch 2/25
 - 230s - loss: 0.1244 - crf_marginal_accuracy: 0.6781 - val_loss: 0.1633 - val_crf_marginal_accuracy: 0.6888

Epoch 00002: val_crf_marginal_accuracy improved from 0.68600 to 0.68883, saving model to model1.h5
Epoch 3/25
 - 230s - loss: 0.1226 - crf_marginal_accuracy: 0.6844 - val_loss: 0.0732 - val_crf_marginal_accuracy: 0.7319

Epoch 00003: val_crf_marginal_accuracy improved from 0.68883 to 0.73193, saving model to model1.h5
Epoch 4/25
 - 230s - loss: 0.1100 - crf_marginal_accuracy: 0.7018 - val_loss: 0.2612 - val_crf_marginal_accuracy: 0.7610

Epoch 00004: val_crf_marginal_accuracy improved from 0.73193 to 0.76097, saving model to model1.h5
Epoch 5/25
 - 230s - loss: 0.1025 - crf_marginal_accuracy: 0.7317 - val_loss: 0.0653 - va

100%|██████████| 2/2 [04:09<00:00, 124.84s/it]


{'20fold\\SC4001E0.npz': <numpy.lib.npyio.NpzFile object at 0x000001EEEEB1F448>, '20fold\\SC4002E0.npz': <numpy.lib.npyio.NpzFile object at 0x000001EEEEB1F888>}
Seq Test f1 score (deep): 0.8050763759334203 
Seq Test accuracy score(deeo) : 0.8673780487804879 
              precision    recall  f1-score   support

           0       0.92      0.92      0.92       371
           1       0.51      0.40      0.45       117
           2       0.84      0.90      0.87       623
           3       0.97      0.83      0.89       517
           4       0.82      0.96      0.89       340

    accuracy                           0.87      1968
   macro avg       0.81      0.80      0.81      1968
weighted avg       0.87      0.87      0.86      1968

[[341  19   1   0  10]
 [ 24  47  10   0  36]
 [  1  21 563  14  24]
 [  2   0  86 429   0]
 [  1   5   7   0 327]]
Epoch 1/25
 - 228s - loss: 0.2402 - crf_marginal_accuracy: 0.5562 - val_loss: 0.2065 - val_crf_marginal_accuracy: 0.4764

Epoch 00001: v

100%|██████████| 2/2 [05:38<00:00, 169.29s/it]


{'20fold\\SC4011E0.npz': <numpy.lib.npyio.NpzFile object at 0x000001EEEEB1F348>, '20fold\\SC4012E0.npz': <numpy.lib.npyio.NpzFile object at 0x000001EEE52FB248>}
Seq Test f1 score (deep): 0.6454703495680274 
Seq Test accuracy score(deeo) : 0.746177370030581 
              precision    recall  f1-score   support

           0       0.58      0.50      0.54       319
           1       0.48      0.30      0.37       201
           2       0.86      0.87      0.87      1222
           3       0.71      0.93      0.81       201
           4       0.60      0.69      0.64       346

    accuracy                           0.75      2289
   macro avg       0.65      0.66      0.65      2289
weighted avg       0.74      0.75      0.74      2289

[[ 159   14   99    2   45]
 [  40   61   46    1   53]
 [  12   15 1061   72   62]
 [   0    0   14  187    0]
 [  61   37    8    0  240]]
Epoch 1/25
 - 235s - loss: 0.2319 - crf_marginal_accuracy: 0.5892 - val_loss: 0.1312 - val_crf_marginal_accuracy

...

{'20fold\\SC4181E0.npz': <numpy.lib.npyio.NpzFile object at 0x000001F24D7B4908>, '20fold\\SC4182E0.npz': <numpy.lib.npyio.NpzFile object at 0x000001F24D7B4548>}
Seq Test f1 score (deep): 0.5940909882110719 
Seq Test accuracy score(deeo) : 0.6656050955414012 
              precision    recall  f1-score   support

           0       0.56      0.98      0.71       285
           1       0.57      0.13      0.22       180
           2       0.69      0.66      0.68       678
           3       0.99      0.64      0.78       507
           4       0.48      0.75      0.58       234

    accuracy                           0.67      1884
   macro avg       0.66      0.63      0.59      1884
weighted avg       0.71      0.67      0.65      1884

[[278   2   1   0   4]
 [ 79  24  28   0  49]
 [ 74  11 450   4 139]
 [ 14   0 166 327   0]
 [ 49   5   5   0 175]]
Epoch 1/25
 - 238s - loss: 0.2485 - crf_marginal_accuracy: 0.5740 - val_loss: 0.1696 - val_crf_marginal_accuracy: 0.6837

Epoch 00001: v

100%|██████████| 2/2 [09:09<00:00, 274.64s/it]


{'20fold\\SC4191E0.npz': <numpy.lib.npyio.NpzFile object at 0x000001F283CC7548>, '20fold\\SC4192E0.npz': <numpy.lib.npyio.NpzFile object at 0x000001F283CC7348>}
Seq Test f1 score (deep): 0.6763509915461556 
Seq Test accuracy score(deeo) : 0.7582769668921324 
              precision    recall  f1-score   support

           0       0.86      0.56      0.68       564
           1       0.48      0.21      0.29       190
           2       0.74      0.93      0.83      1267
           3       0.81      0.91      0.86       170
           4       0.76      0.71      0.73       618

    accuracy                           0.76      2809
   macro avg       0.73      0.66      0.68      2809
weighted avg       0.76      0.76      0.74      2809

[[ 318   23  170    0   53]
 [  38   39   66    0   47]
 [   2    7 1183   36   39]
 [   0    0   16  154    0]
 [  12   13  157    0  436]]
Epoch 1/25
 - 238s - loss: 0.3009 - crf_marginal_accuracy: 0.5278 - val_loss: 0.1886 - val_crf_marginal_accurac

100%|██████████| 1/1 [01:25<00:00, 85.67s/it]

{'20fold\\SC4231E0.npz': <numpy.lib.npyio.NpzFile object at 0x000001F22D9F6DC8>}
Seq Test f1 score (deep): 0.8001588830857124 
Seq Test accuracy score(deeo) : 0.8628404669260701 
              precision    recall  f1-score   support

           0       0.89      0.90      0.90       155
           1       0.46      0.44      0.45        57
           2       0.94      0.83      0.88       497
           3       0.81      0.99      0.89       147
           4       0.82      0.94      0.88       172

    accuracy                           0.86      1028
   macro avg       0.79      0.82      0.80      1028
weighted avg       0.87      0.86      0.86      1028

[[140   6   5   2   2]
 [ 14  25   9   0   9]
 [  3  23 414  33  24]
 [  0   0   1 146   0]
 [  0   0  10   0 162]]
Seq Test f1 score : 0.7113416722750381 
Seq Test accuracy score : 0.7873688191358609 
              precision    recall  f1-score   support

           0       0.86      0.77      0.81      8285
           1       0.

In [35]:
np.unique(b[0,:,0])


array([1, 2, 4])